In [2]:
import pandas as pd
from datetime import datetime, timedelta
import sys
sys.path.append('/home/lowinli/ditu/test/')
from utility import items, pro_df
import pymongo
db= pymongo.MongoClient('localhost',27017).Emotion_Count_Db

def group_week(df):
    df['datetime']=pd.to_datetime(df.index)
    df['week'] = df['datetime'].apply(lambda x: x-timedelta(days = x.weekday()))
    datas = []
    for day, day_df in df.groupby('week'):
        data = [day]
        for item in items:
            data.append(day_df[item].sum())
        datas.append(data)
    new_df = pd.DataFrame(datas,columns=['week']+items).iloc[1:-1]
    new_df.index = pd.to_datetime(new_df['week'])
    return new_df


def  gen_data():
    datas=[]
    df = pd.DataFrame(db['quanbu'].find())
    df = group_week(df)
    for dt in df['datetime'].tolist():
        datas.append({'time':dt.strftime('%Y%m%d'),'data':[]})
    for record in pro_df.to_dict(orient='records'):
        df = pd.DataFrame(db[record['e']].find())
        df = group_week(df)
        i=0
        for x in df.to_dict(otient='records'):
            pro_d = {'name':record['c'],'value':[x['全部'],0,record['c']]}
            datas[i]['data'] .append(pro_d)
    return datas
if __name__ == "__main__":
    datas = gen_data()
    datas


KeyError: 'datetime'